In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
import os

Using TensorFlow backend.


In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount  =True)
if os.getcwd()!= "./drive/My Drive/deep_learning":
  os.chdir("./drive/My Drive/deep_learning")

Mounted at /content/drive/


In [0]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.95)
config = tf.ConfigProto(log_device_placement=True,gpu_options=gpu_options)
sess = tf.Session(config = config)
keras.backend.set_session(sess)

In [0]:
batch_size = 32 
epochs = 200
num_classes = 10
depth = 20
subtract_pixel_mean = True

In [0]:
model_type = 'ResNet%d' % (depth)

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [8]:
input_shape = x_train.shape[1:]
input_shape

(32, 32, 3)

In [0]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [0]:
if subtract_pixel_mean:
  x_train_mean = np.mean(x_train, axis=0)
  x_train -= x_train_mean
  x_test -= x_train_mean

In [11]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def lr_schedule(epoch):
  lr = 1e-3
  if epoch > 180:
      lr *= 0.5e-3
  elif epoch > 160:
      lr *= 1e-3
  elif epoch > 120:
      lr *= 1e-2
  elif epoch > 80:
      lr *= 1e-1
  print('Learning rate: ', lr)
  return lr

In [0]:
def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu', batch_normalization=True, conv_first=True):
  
  conv = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')

  x = inputs
  if conv_first:
      x = conv(x)
      if batch_normalization:
          x = BatchNormalization()(x)
      if activation is not None:
          x = Activation(activation)(x)
  else:
      if batch_normalization:
          x = BatchNormalization()(x)
      if activation is not None:
          x = Activation(activation)(x)
      x = conv(x)
  return x

In [0]:
def resnet_v1(input_shape, depth, num_classes=10):
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,num_filters=num_filters,strides=strides)
            y = resnet_layer(inputs=y,num_filters=num_filters,activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,num_filters=num_filters,kernel_size=1,strides=strides,activation=None,batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
            x = Dropout(rate=0.25)(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [16]:
model = resnet_v1(input_shape=input_shape, depth=depth)

W0701 13:08:59.492069 140207348557696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0701 13:08:59.497661 140207348557696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0701 13:08:59.503345 140207348557696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0701 13:08:59.530490 140207348557696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0701 13:08:59.531512 1402073485

In [17]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=lr_schedule(0)),metrics=['accuracy'])

W0701 13:09:04.917957 140207348557696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Learning rate:  0.001


In [18]:
model.summary()
print(model_type)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models_with_dropout')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [0]:
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc',verbose=1,save_best_only=True)

In [0]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [0]:
callbacks = [checkpoint, lr_scheduler]

In [23]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test, y_test),shuffle=True,callbacks=callbacks)

W0701 13:09:30.220890 140207348557696 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/200
Learning rate:  0.001
50000/50000 [==============================] - 47s 943us/step - loss: 1.6247 - acc: 0.3948 - val_loss: 1.8751 - val_acc: 0.4223

Epoch 00001: val_acc improved from -inf to 0.42230, saving model to /content/drive/My Drive/deep_learning/saved_models_with_dropout/cifar10_ResNet20_model.001.h5
Epoch 2/200
Learning rate:  0.001
50000/50000 [==============================] - 41s 815us/step - loss: 1.2443 - acc: 0.5516 - val_loss: 1.4252 - val_acc: 0.5311

Epoch 00002: val_acc improved from 0.42230 to 0.53110, saving model to /content/drive/My Drive/deep_learning/saved_models_with_dropout/cifar10_ResNet20_model.002.h5
Epoch 3/200
Learning rate:  0.001
50000/50000 [==============================] - 41s 818us/step - loss: 1.0849 - acc: 0.6131 - val_loss: 1.0165 - val_acc: 0.6404

Epoch 00003: val_acc improved from 0.53110 to 0.64040, saving model to /content/drive/My Drive/deep_learning/saved_models_with_dropout

In [24]:
with open("./model.json", "w") as json_file:
  json_file.write(model.to_json())
  model.save_weights("./model_weights.h5")
  print("model saved model to disk")

model saved model to disk


In [26]:
loss, acc = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 137us/step


In [0]:
print("loss: ",loss,", Accura")